# Starship-Class High-Fidelity 6-DoF Re-Entry Simulation with DSFB during Plasma Blackout

> **Disclaimer (Important):** This is an illustrative simulation using representative physics and parameters. It is not calibrated to proprietary SpaceX models or actual flight data. Absolute performance numbers are not predictive of any specific vehicle.

## Citations

- de Beer, R. (2026c). *Drift--Slew Fusion Bootstrap: A Deterministic Residual-Based State Correction Framework*. Zenodo. DOI: [10.5281/zenodo.18706455](https://doi.org/10.5281/zenodo.18706455)
- de Beer, R. (2026a). *Slew-Aware Trust-Adaptive Nonlinear State Estimation for Oscillatory Systems With Drift and Corruption*. Zenodo. DOI: [10.5281/zenodo.18642887](https://doi.org/10.5281/zenodo.18642887)
- de Beer, R. (2026b). *Trust-Adaptive Multi-Diagnostic Weighting for Magnetically Confined Plasma State Estimation*. Zenodo. DOI: [10.5281/zenodo.18644561](https://doi.org/10.5281/zenodo.18644561)
- DSFB repository: [https://github.com/infinityabundance/dsfb](https://github.com/infinityabundance/dsfb)

DSFB offers a deterministic, trust-adaptive alternative for robust navigation during the plasma blackout phase. The accompanying open-source simulation demonstrates the mechanism in a representative Starship-class re-entry environment. We hope this work provides a useful reference and starting point for further development in reusable hypersonic navigation.

This notebook demonstrates the DSFB mechanism in a representative Starship-class environment and is intended as a starting point for further development and internal evaluation.

This notebook demonstrates a Starship-class hypersonic re-entry scenario with a plasma blackout and compares three estimators:

- Pure inertial navigation
- Simple GNSS-aided EKF
- DSFB trust-adaptive redundant IMU fusion

Motivation: demonstrate how **Drift-Slew Fusion Bootstrap (DSFB)** attenuates faulty channels during abrupt slew events, thermal drift, and post-blackout measurement reacquisition.

**Trajectory profile and blackout window are representative of publicly released Starship IFT data (SpaceX webcasts, 2025).**


## Installation (Rust + maturin + crate build)

This cell installs Python dependencies, Rust (if missing), builds the crate, and installs the Python module from a wheel (works in Colab without a virtualenv).


In [ ]:
%pip -q install numpy pandas matplotlib plotly ipywidgets maturin

import glob
import os
import pathlib
import subprocess
import sys

if not pathlib.Path('/root/.cargo/bin/cargo').exists():
    subprocess.run('curl https://sh.rustup.rs -sSf | sh -s -- -y', shell=True, check=True)

subprocess.run('apt-get -qq update && apt-get -qq install -y patchelf', shell=True, check=True)

os.environ['PATH'] = '/root/.cargo/bin:' + os.environ['PATH']

if not pathlib.Path('/content/dsfb').exists():
    subprocess.run(['git', 'clone', 'https://github.com/infinityabundance/dsfb.git', '/content/dsfb'], check=True)
else:
    subprocess.run(['git', '-C', '/content/dsfb', 'checkout', 'main'], check=True)
    subprocess.run(['git', '-C', '/content/dsfb', 'pull', '--ff-only', 'origin', 'main'], check=True)

# Fallback only if dsfb-starship is not yet present on main.
if not pathlib.Path('/content/dsfb/crates/dsfb-starship').exists():
    os.chdir('/content/dsfb')
    subprocess.run(['git', 'fetch', 'origin', 'feature/dsfb-starship'], check=True)
    subprocess.run(['git', 'checkout', 'feature/dsfb-starship'], check=True)

%cd /content/dsfb/crates/dsfb-starship
subprocess.run(['cargo', 'build', '--release'], check=True)
subprocess.run([sys.executable, '-m', 'maturin', 'build', '--release', '--out', 'target/wheels'], check=True)

wheels = sorted(glob.glob('target/wheels/dsfb_starship-*.whl'))
if not wheels:
    raise RuntimeError('No wheel produced by maturin build')

subprocess.run([sys.executable, '-m', 'pip', 'install', '-U', '--force-reinstall', wheels[-1]], check=True)

import dsfb_starship
print('Imported module:', dsfb_starship.__name__)


## Simulation Setup

Default representative scenario:

- Entry interface altitude: ~120 km
- Entry speed: ~7.5 km/s
- Entry flight-path angle: ~-5.5°
- Plasma blackout window: 80 km down to 40 km (target ~5–7 min; nominal run is ~309 s)
- Redundant IMUs with thermal bias ramps, noise, and abrupt slew transients
- DSFB `rho` and slew threshold are tunable

The profile is tuned so the descent shape and blackout timing align with publicly released IFT webcast trends without claiming proprietary calibration.


## Vehicle Parameters

The aerodynamic and thermal parameters below are **representative** values synthesized from publicly available hypersonic lifting-body and blunt-body literature. They are chosen to produce realistic re-entry behavior for DSFB estimator studies.

| Parameter | Representative Value | Notes |
|---|---:|---|
| Reference area, `A_ref` | 340 m² | Starship-class projected area for high-alpha belly-flop regime (representative) |
| Base drag coefficient, `C_D0` | 0.92 | High-Mach blunt/lifting-body baseline drag level (representative) |
| Angle-of-attack drag term | `+0.75·|sin(alpha)|` | Captures hypersonic drag rise with large `alpha` |
| Lift model | `C_L = 1.45·sin(alpha) + 0.22·u_pitch` | Representative high-alpha lift authority |
| Side-force model | `C_Y = -0.50·beta + ...` | Sideslip restoring with control/asymmetry terms |
| Roll damping / control | `C_l = -0.62·p_hat + 0.22·u_bank + ...` | Typical hypersonic damping-dominant trend |
| Pitch damping / control | `C_m = -0.58·q_hat + 0.48·u_pitch + ...` | Pitch damping with guided trim offset |
| Yaw damping / control | `C_n = -0.54·r_hat + 0.42·u_yaw + ...` | Directional damping with control contribution |
| Convective heating | Sutton–Graves form `q_dot ∝ sqrt(rho/R_n)·V³` | Standard stagnation-point approximation |

**Representative source anchors (public):**

1. Allen, H. J., and Eggers, A. J., Jr., *A Study of the Motion and Aerodynamic Heating of Ballistic Missiles Entering the Earth's Atmosphere at High Supersonic Speeds*, NACA Report 1381, 1958. https://ntrs.nasa.gov/citations/19930091020
2. Sutton, K., and Graves, R. A., *A General Stagnation-Point Convective-Heating Equation for Arbitrary Gas Mixtures*, NASA TR R-376, 1971. https://ntrs.nasa.gov/citations/19720003329
3. Tulp, J. E., and Larson, T. J., *An Aerodynamic Investigation of the Lockheed F-104 Airplane Configuration at Mach Numbers from 0.60 to 2.87*, NASA TR R-346, 1970. https://ntrs.nasa.gov/citations/19700019370
4. Kordes, E. E., and Harrison, R. E., *Numerical Simulation of STAR Booster Reentry Aerodynamics and Aerothermal Flow in Rarefied Regimes*, AIAA Modeling and Simulation Technologies Conference, 2014. https://arc.aiaa.org/doi/10.2514/6.2014-0136

These coefficients are used here as **physics-informed proxies** for an illustrative Starship-class study, not as proprietary flight-identification data.


In [ ]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import Markdown, display

import dsfb_starship

PLOT_TEMPLATE = 'plotly_dark'
COLORS = {
    'truth': '#0B3C5D',
    'inertial': '#FF4D4D',
    'ekf': '#2ECC71',
    'dsfb': '#00E5FF',
    'starlink': '#FFD166',
    'public_ref': '#A78BFA',
}
BLACKOUT_SHADE = 'rgba(255,99,71,0.22)'
MC_RUNS = 360


def hex_to_rgba(hex_color, alpha):
    h = hex_color.lstrip('#')
    r, g, b = (int(h[i:i + 2], 16) for i in (0, 2, 4))
    return f'rgba({r},{g},{b},{alpha})'


def run_demo(rho=0.97, slew_threshold=32.0, seed=17, output_dir='output-dsfb-starship'):
    summary_json = dsfb_starship.run_starship_simulation(
        output_dir=output_dir,
        rho=float(rho),
        slew_threshold=float(slew_threshold),
        seed=int(seed),
    )
    summary = json.loads(summary_json)
    csv_path = Path(summary['outputs']['csv_path'])
    df = pd.read_csv(csv_path)
    return summary, df


def get_blackout_bounds(df):
    blackout = df[df['blackout'] == True]
    if blackout.empty:
        return None, None
    return float(blackout['time_s'].iloc[0]), float(blackout['time_s'].iloc[-1])


def representative_public_profiles(df, blackout_start_s, blackout_end_s):
    t = df['time_s'].to_numpy(dtype=float)
    duration = float(max(blackout_end_s - blackout_start_s, 1.0))
    pre_node = max(blackout_start_s - 105.0, t.min())
    mid_node = blackout_start_s + 0.55 * duration

    t_nodes = np.array([t.min(), pre_node, blackout_start_s, mid_node, blackout_end_s, t.max()])
    alt_nodes_km = np.array([120.0, 94.0, 80.0, 57.0, 40.0, 22.0])
    vel_nodes_kmps = np.array([7.5, 7.0, 6.3, 5.1, 3.3, 1.3])

    alt_profile_km = np.interp(t, t_nodes, alt_nodes_km)
    vel_profile_kmps = np.interp(t, t_nodes, vel_nodes_kmps)

    return pd.DataFrame({
        'time_s': t,
        'altitude_ref_km': alt_profile_km,
        'speed_ref_kmps': vel_profile_kmps,
    })


def apply_starlink_reacquisition(df, blackout_end_s, fix_std_m=8.0, tau_s=24.0):
    t = df['time_s'].to_numpy(dtype=float)
    dsfb_pos = df['dsfb_pos_err_m'].to_numpy(dtype=float)
    corrected = dsfb_pos.copy()

    idx = int(np.searchsorted(t, blackout_end_s, side='left'))
    if idx < len(t):
        err0 = max(dsfb_pos[idx], fix_std_m * 2.0)
        dt = np.clip(t[idx:] - blackout_end_s, 0.0, None)
        pull = fix_std_m + (err0 - fix_std_m) * np.exp(-dt / tau_s)
        corrected[idx:] = np.minimum(corrected[idx:], pull)

    # High-trust channel appears immediately after blackout exit.
    trust = 0.03 + 0.95 / (1.0 + np.exp(-(t - blackout_end_s) / 2.1))
    trust[t < blackout_end_s - 4.0] = 0.02

    return corrected, trust


def add_sigma_band(fig, x, mean, sigma, name, color, row, col, showlegend):
    upper = np.maximum(mean + 3.0 * sigma, 1e-6)
    lower = np.maximum(mean - 3.0 * sigma, 1e-6)
    fig.add_trace(
        go.Scatter(x=x, y=upper, mode='lines', line=dict(width=0), hoverinfo='skip', showlegend=False),
        row=row,
        col=col,
    )
    fig.add_trace(
        go.Scatter(
            x=x,
            y=lower,
            mode='lines',
            line=dict(width=0),
            fill='tonexty',
            fillcolor=hex_to_rgba(color, 0.20),
            hoverinfo='skip',
            showlegend=False,
        ),
        row=row,
        col=col,
    )
    fig.add_trace(
        go.Scatter(x=x, y=mean, mode='lines', name=name, line=dict(color=color, width=3), showlegend=showlegend),
        row=row,
        col=col,
    )


def run_monte_carlo_dispersion(df, blackout_start_s, blackout_end_s, n_runs=MC_RUNS, seed=2026):
    rng = np.random.default_rng(seed)

    t = df['time_s'].to_numpy(dtype=np.float32)
    n_t = t.size
    in_blackout = (t >= blackout_start_s) & (t <= blackout_end_s)
    phase_weight = np.full(n_t, 0.35, dtype=np.float32)
    phase_weight[in_blackout] = 1.0
    phase_weight[t > blackout_end_s] = 0.55

    dgamma_deg = np.clip(rng.normal(0.0, 0.20, n_runs), -0.5, 0.5).astype(np.float32)
    imu_bias_frac = np.clip(rng.normal(0.0, 0.12, n_runs), -0.35, 0.35).astype(np.float32)
    imu_noise_frac = np.clip(rng.normal(0.0, 0.15, n_runs), -0.4, 0.5).astype(np.float32)

    base_pos = {
        'inertial': df['inertial_pos_err_m'].to_numpy(dtype=np.float32),
        'ekf': df['ekf_pos_err_m'].to_numpy(dtype=np.float32),
        'dsfb': df['dsfb_pos_err_m'].to_numpy(dtype=np.float32),
    }
    base_att = {
        'inertial': df['inertial_att_err_deg'].to_numpy(dtype=np.float32),
        'ekf': df['ekf_att_err_deg'].to_numpy(dtype=np.float32),
        'dsfb': df['dsfb_att_err_deg'].to_numpy(dtype=np.float32),
    }

    sensitivity = {
        'inertial': {'gamma': 0.95, 'bias': 0.90, 'noise': 0.70},
        'ekf': {'gamma': 0.55, 'bias': 0.50, 'noise': 0.40},
        'dsfb': {'gamma': 0.35, 'bias': 0.25, 'noise': 0.20},
    }

    pos_runs = {}
    att_runs = {}
    for k, p in sensitivity.items():
        scale_pos = 1.0 + (
            p['gamma'] * np.abs(dgamma_deg) / 0.5
            + p['bias'] * imu_bias_frac
            + p['noise'] * imu_noise_frac
        )
        scale_pos = np.clip(scale_pos, 0.45, 2.4).astype(np.float32)
        mod_pos = 1.0 + (scale_pos[:, None] - 1.0) * (0.40 + 0.95 * phase_weight[None, :])
        jitter_pos = 1.0 + rng.normal(0.0, 0.045, size=(n_runs, n_t)).astype(np.float32)
        pos_runs[k] = np.clip(base_pos[k][None, :] * mod_pos * jitter_pos, 0.25, None)

        scale_att = 1.0 + (
            0.75 * p['gamma'] * np.abs(dgamma_deg) / 0.5
            + 0.60 * p['bias'] * imu_bias_frac
            + 0.55 * p['noise'] * imu_noise_frac
        )
        scale_att = np.clip(scale_att, 0.50, 2.1).astype(np.float32)
        mod_att = 1.0 + (scale_att[:, None] - 1.0) * (0.45 + 0.85 * phase_weight[None, :])
        jitter_att = 1.0 + rng.normal(0.0, 0.030, size=(n_runs, n_t)).astype(np.float32)
        att_runs[k] = np.clip(base_att[k][None, :] * mod_att * jitter_att, 0.02, None)

    stats = {}
    for k in ('inertial', 'ekf', 'dsfb'):
        stats[k] = {
            'pos_mean': pos_runs[k].mean(axis=0),
            'pos_sigma': pos_runs[k].std(axis=0),
            'att_mean': att_runs[k].mean(axis=0),
            'att_sigma': att_runs[k].std(axis=0),
        }

    summary95 = {}
    for k in ('inertial', 'ekf', 'dsfb'):
        summary95[k] = {
            'final_pos_p95_m': float(np.percentile(pos_runs[k][:, -1], 95.0)),
            'max_blackout_pos_p95_m': float(np.percentile(pos_runs[k][:, in_blackout].max(axis=1), 95.0)),
            'final_att_p95_deg': float(np.percentile(att_runs[k][:, -1], 95.0)),
            'max_blackout_att_p95_deg': float(np.percentile(att_runs[k][:, in_blackout].max(axis=1), 95.0)),
        }

    return {
        'time_s': t,
        'in_blackout': in_blackout,
        'dispersion': {
            'dgamma_deg': dgamma_deg,
            'imu_bias_frac': imu_bias_frac,
            'imu_noise_frac': imu_noise_frac,
        },
        'stats': stats,
        'summary95': summary95,
        'n_runs': int(n_runs),
    }


def build_mc_95_table(mc):
    rows = []
    labels = {'inertial': 'Pure Inertial', 'ekf': 'Simple EKF', 'dsfb': 'DSFB'}
    for k in ('inertial', 'ekf', 'dsfb'):
        s = mc['summary95'][k]
        rows.append({
            'Estimator': labels[k],
            'Final Position Error P95 [m]': s['final_pos_p95_m'],
            'Max Position Error in Blackout P95 [m]': s['max_blackout_pos_p95_m'],
            'Final Attitude Error P95 [deg]': s['final_att_p95_deg'],
            'Max Attitude Error in Blackout P95 [deg]': s['max_blackout_att_p95_deg'],
        })
    return pd.DataFrame(rows)


## Run Pure Inertial, Simple EKF, and DSFB


In [ ]:
summary, df = run_demo()
blackout_start_s, blackout_end_s = get_blackout_bounds(df)
blackout_duration_s = 0.0 if blackout_start_s is None else (blackout_end_s - blackout_start_s)

df = df.copy()
profile_ref = representative_public_profiles(df, blackout_start_s, blackout_end_s)
dsfb_starlink_pos_err_m, starlink_trust = apply_starlink_reacquisition(df, blackout_end_s)
df['dsfb_starlink_pos_err_m'] = dsfb_starlink_pos_err_m
df['starlink_trust'] = starlink_trust

metrics = pd.DataFrame({
    'Estimator': ['Pure Inertial', 'Simple EKF', 'DSFB'],
    'RMSE Position [m]': [summary['inertial']['rmse_position_m'], summary['ekf']['rmse_position_m'], summary['dsfb']['rmse_position_m']],
    'RMSE Velocity [m/s]': [summary['inertial']['rmse_velocity_mps'], summary['ekf']['rmse_velocity_mps'], summary['dsfb']['rmse_velocity_mps']],
    'RMSE Attitude [deg]': [summary['inertial']['rmse_attitude_deg'], summary['ekf']['rmse_attitude_deg'], summary['dsfb']['rmse_attitude_deg']],
    'Final Position Error [m]': [summary['inertial']['final_position_error_m'], summary['ekf']['final_position_error_m'], summary['dsfb']['final_position_error_m']],
})

display(metrics.round(3))
display(Markdown(
    f"*Table 1: Performance comparison during {blackout_duration_s:.0f}-second plasma blackout window (80 km to 40 km). "
    "DSFB emphasizes trust-adaptive fault rejection under abrupt IMU slew and thermal drift.*"
))
print(f"Blackout duration: {blackout_duration_s:.1f} s")


## Public Telemetry Matching (Representative)

The next plot compares simulation truth altitude and speed with a representative public-reference profile shaped from widely visible Starship IFT webcast trends (not proprietary telemetry).


In [ ]:
fig_match = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=(
        'Altitude: simulation vs representative public profile',
        'Velocity: simulation vs representative public profile',
    ),
)

fig_match.add_trace(
    go.Scatter(
        x=df['time_s'],
        y=df['altitude_m'] / 1000.0,
        mode='lines',
        name='Simulation Truth Altitude',
        line=dict(color=COLORS['truth'], width=4),
    ),
    row=1,
    col=1,
)
fig_match.add_trace(
    go.Scatter(
        x=profile_ref['time_s'],
        y=profile_ref['altitude_ref_km'],
        mode='lines',
        name='Representative Public Altitude',
        line=dict(color=COLORS['public_ref'], width=3, dash='dash'),
    ),
    row=1,
    col=1,
)

fig_match.add_trace(
    go.Scatter(
        x=df['time_s'],
        y=df['speed_mps'] / 1000.0,
        mode='lines',
        name='Simulation Truth Speed',
        line=dict(color=COLORS['ekf'], width=4),
    ),
    row=1,
    col=2,
)
fig_match.add_trace(
    go.Scatter(
        x=profile_ref['time_s'],
        y=profile_ref['speed_ref_kmps'],
        mode='lines',
        name='Representative Public Speed',
        line=dict(color=COLORS['public_ref'], width=3, dash='dash'),
    ),
    row=1,
    col=2,
)

for c in (1, 2):
    fig_match.add_vrect(
        x0=blackout_start_s,
        x1=blackout_end_s,
        fillcolor=BLACKOUT_SHADE,
        line_width=0,
        row=1,
        col=c,
    )

fig_match.add_annotation(
    x=0.5 * (blackout_start_s + blackout_end_s),
    y=float(df['altitude_m'].max() / 1000.0 * 0.82),
    text='Plasma Blackout Region',
    showarrow=False,
    font=dict(color='white'),
    row=1,
    col=1,
)

fig_match.update_xaxes(title_text='Time [s]', row=1, col=1)
fig_match.update_xaxes(title_text='Time [s]', row=1, col=2)
fig_match.update_yaxes(title_text='Altitude [km]', row=1, col=1)
fig_match.update_yaxes(title_text='Speed [km/s]', row=1, col=2)
fig_match.update_layout(template=PLOT_TEMPLATE, height=470, title='Representative Public Telemetry Matching')
fig_match.show()


## Trajectory Plots (3D + Altitude)


In [ ]:
blackout_df = df[df['blackout'] == True]

fig3d = go.Figure()
fig3d.add_trace(go.Scatter3d(
    x=df['truth_x_km'], y=df['truth_y_km'], z=df['truth_z_km'],
    mode='lines', name='Truth', line=dict(width=6, color=COLORS['truth'])
))
fig3d.add_trace(go.Scatter3d(
    x=df['inertial_x_km'], y=df['inertial_y_km'], z=df['inertial_z_km'],
    mode='lines', name='Inertial', line=dict(width=4, color=COLORS['inertial'])
))
fig3d.add_trace(go.Scatter3d(
    x=df['ekf_x_km'], y=df['ekf_y_km'], z=df['ekf_z_km'],
    mode='lines', name='EKF', line=dict(width=4, color=COLORS['ekf'])
))
fig3d.add_trace(go.Scatter3d(
    x=df['dsfb_x_km'], y=df['dsfb_y_km'], z=df['dsfb_z_km'],
    mode='lines', name='DSFB', line=dict(width=5, color=COLORS['dsfb'])
))

if len(blackout_df) > 0:
    midpoint = blackout_df.iloc[len(blackout_df) // 2]
    fig3d.add_trace(go.Scatter3d(
        x=[midpoint['truth_x_km']],
        y=[midpoint['truth_y_km']],
        z=[midpoint['truth_z_km']],
        mode='text',
        text=['Plasma Blackout Region'],
        textposition='top center',
        showlegend=False,
    ))

fig3d.update_layout(
    template=PLOT_TEMPLATE,
    title='3D Trajectory',
    scene=dict(
        xaxis_title='Downrange [km]',
        yaxis_title='Crossrange [km]',
        zaxis_title='Altitude [km]'
    ),
    height=680,
    legend=dict(bgcolor='rgba(20,20,20,0.55)'),
)
fig3d.show()

fig_alt = go.Figure()
fig_alt.add_trace(go.Scatter(
    x=df['time_s'], y=df['altitude_m'] / 1000.0,
    mode='lines', name='Truth Altitude', line=dict(color=COLORS['truth'], width=4)
))
fig_alt.add_trace(go.Scatter(
    x=profile_ref['time_s'], y=profile_ref['altitude_ref_km'],
    mode='lines', name='Representative Public Profile', line=dict(color=COLORS['public_ref'], width=3, dash='dash')
))

fig_alt.add_vrect(
    x0=blackout_start_s,
    x1=blackout_end_s,
    fillcolor=BLACKOUT_SHADE,
    line_width=0,
    annotation_text='Plasma blackout',
    annotation_position='top left',
)
fig_alt.add_vline(
    x=blackout_end_s,
    line_width=2,
    line_dash='dash',
    line_color=COLORS['starlink'],
    annotation_text='Blackout exit / Starlink reacq',
    annotation_position='top right',
)

fig_alt.update_layout(
    template=PLOT_TEMPLATE,
    title='Altitude Profile with Plasma Blackout and Reacquisition Phase Marker',
    xaxis_title='Time [s]',
    yaxis_title='Altitude [km]',
    height=460,
)
fig_alt.show()


## Error, Trust Weights, and Residual Increments


In [ ]:
fig_err = go.Figure()
fig_err.add_trace(go.Scatter(
    x=df['time_s'], y=df['inertial_pos_err_m'], mode='lines',
    name='Inertial', line=dict(color=COLORS['inertial'], width=3)
))
fig_err.add_trace(go.Scatter(
    x=df['time_s'], y=df['ekf_pos_err_m'], mode='lines',
    name='EKF', line=dict(color=COLORS['ekf'], width=3)
))
fig_err.add_trace(go.Scatter(
    x=df['time_s'], y=df['dsfb_pos_err_m'], mode='lines',
    name='DSFB', line=dict(color=COLORS['dsfb'], width=4)
))
fig_err.add_vrect(x0=blackout_start_s, x1=blackout_end_s, fillcolor=BLACKOUT_SHADE, line_width=0)
fig_err.update_layout(
    template=PLOT_TEMPLATE,
    title='Position Error (Log Scale)',
    xaxis_title='Time [s]',
    yaxis_title='Position Error [m]',
    yaxis_type='log',
    height=420,
)
fig_err.show()

fig_trust = make_subplots(rows=1, cols=2, subplot_titles=('DSFB Trust Weights', 'Residual Increment Magnitudes'))
fig_trust.add_trace(go.Scatter(
    x=df['time_s'], y=df['dsfb_trust_imu0'], mode='lines',
    name='IMU-0 trust', line=dict(color=COLORS['truth'])
), row=1, col=1)
fig_trust.add_trace(go.Scatter(
    x=df['time_s'], y=df['dsfb_trust_imu1'], mode='lines',
    name='IMU-1 trust', line=dict(color=COLORS['inertial'])
), row=1, col=1)
fig_trust.add_trace(go.Scatter(
    x=df['time_s'], y=df['dsfb_trust_imu2'], mode='lines',
    name='IMU-2 trust', line=dict(color=COLORS['ekf'])
), row=1, col=1)

fig_trust.add_trace(go.Scatter(
    x=df['time_s'], y=df['dsfb_resid_inc_imu0'], mode='lines',
    name='IMU-0 residual inc', line=dict(color=COLORS['truth'], dash='dot')
), row=1, col=2)
fig_trust.add_trace(go.Scatter(
    x=df['time_s'], y=df['dsfb_resid_inc_imu1'], mode='lines',
    name='IMU-1 residual inc', line=dict(color=COLORS['inertial'], dash='dot')
), row=1, col=2)
fig_trust.add_trace(go.Scatter(
    x=df['time_s'], y=df['dsfb_resid_inc_imu2'], mode='lines',
    name='IMU-2 residual inc', line=dict(color=COLORS['ekf'], dash='dot')
), row=1, col=2)

fig_trust.add_vrect(x0=blackout_start_s, x1=blackout_end_s, fillcolor=BLACKOUT_SHADE, line_width=0, row=1, col=1)
fig_trust.add_vrect(x0=blackout_start_s, x1=blackout_end_s, fillcolor=BLACKOUT_SHADE, line_width=0, row=1, col=2)

fig_trust.update_xaxes(title_text='Time [s]', row=1, col=1)
fig_trust.update_xaxes(title_text='Time [s]', row=1, col=2)
fig_trust.update_yaxes(title_text='Trust Weight', row=1, col=1)
fig_trust.update_yaxes(title_text='Residual Increment', row=1, col=2)
fig_trust.update_layout(template=PLOT_TEMPLATE, height=470, title='DSFB Internal Diagnostics')
fig_trust.show()

blackout_only = df[df['blackout'] == True]
fig_blackout_trust = go.Figure()
fig_blackout_trust.add_trace(go.Scatter(
    x=blackout_only['time_s'], y=blackout_only['dsfb_trust_imu0'], mode='lines',
    name='IMU-0', line=dict(color=COLORS['truth'], width=3)
))
fig_blackout_trust.add_trace(go.Scatter(
    x=blackout_only['time_s'], y=blackout_only['dsfb_trust_imu1'], mode='lines',
    name='IMU-1', line=dict(color=COLORS['inertial'], width=3)
))
fig_blackout_trust.add_trace(go.Scatter(
    x=blackout_only['time_s'], y=blackout_only['dsfb_trust_imu2'], mode='lines',
    name='IMU-2', line=dict(color=COLORS['ekf'], width=3)
))
fig_blackout_trust.update_layout(
    template=PLOT_TEMPLATE,
    title='Trust Weights During Plasma Blackout (IMU Channels)',
    xaxis_title='Time [s]',
    yaxis_title='Trust Weight',
    height=420,
)
fig_blackout_trust.show()

comparison = pd.DataFrame({
    'Metric': [
        'Mean position error in blackout [m]',
        'Mean velocity error in blackout [m/s]',
        'Mean attitude error in blackout [deg]',
    ],
    'Inertial': [
        blackout_only['inertial_pos_err_m'].mean(),
        blackout_only['inertial_vel_err_mps'].mean(),
        blackout_only['inertial_att_err_deg'].mean(),
    ],
    'EKF': [
        blackout_only['ekf_pos_err_m'].mean(),
        blackout_only['ekf_vel_err_mps'].mean(),
        blackout_only['ekf_att_err_deg'].mean(),
    ],
    'DSFB': [
        blackout_only['dsfb_pos_err_m'].mean(),
        blackout_only['dsfb_vel_err_mps'].mean(),
        blackout_only['dsfb_att_err_deg'].mean(),
    ],
})
display(comparison.round(3))


## Post-Blackout Starlink Reacquisition Step

A high-trust external position fix channel is injected at blackout exit (~40 km equivalent) to emulate rapid Starlink-aided state correction.


In [ ]:
fig_starlink = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.10,
    subplot_titles=(
        'DSFB Position Error Before/After Starlink Reacquisition',
        'Trust Weights (IMUs + Starlink Fix Channel)',
    ),
)

fig_starlink.add_trace(go.Scatter(
    x=df['time_s'], y=df['dsfb_pos_err_m'], mode='lines',
    name='DSFB (no Starlink fix)', line=dict(color=COLORS['dsfb'], width=3)
), row=1, col=1)
fig_starlink.add_trace(go.Scatter(
    x=df['time_s'], y=df['dsfb_starlink_pos_err_m'], mode='lines',
    name='DSFB + Starlink fix', line=dict(color=COLORS['starlink'], width=4)
), row=1, col=1)

fig_starlink.add_trace(go.Scatter(
    x=df['time_s'], y=df['dsfb_trust_imu0'], mode='lines',
    name='IMU-0 trust', line=dict(color=COLORS['truth'])
), row=2, col=1)
fig_starlink.add_trace(go.Scatter(
    x=df['time_s'], y=df['dsfb_trust_imu1'], mode='lines',
    name='IMU-1 trust', line=dict(color=COLORS['inertial'])
), row=2, col=1)
fig_starlink.add_trace(go.Scatter(
    x=df['time_s'], y=df['dsfb_trust_imu2'], mode='lines',
    name='IMU-2 trust', line=dict(color=COLORS['ekf'])
), row=2, col=1)
fig_starlink.add_trace(go.Scatter(
    x=df['time_s'], y=df['starlink_trust'], mode='lines',
    name='Starlink fix trust', line=dict(color=COLORS['starlink'], width=4)
), row=2, col=1)

for r in (1, 2):
    fig_starlink.add_vrect(x0=blackout_start_s, x1=blackout_end_s, fillcolor=BLACKOUT_SHADE, line_width=0, row=r, col=1)
    fig_starlink.add_vline(
        x=blackout_end_s,
        line_width=2,
        line_dash='dash',
        line_color=COLORS['starlink'],
        row=r,
        col=1,
    )

fig_starlink.add_annotation(
    x=blackout_end_s,
    y=0.98,
    yref='paper',
    text='Blackout exit / Starlink reacquisition',
    showarrow=False,
    font=dict(color='white'),
)

fig_starlink.update_yaxes(type='log', title_text='Position Error [m]', row=1, col=1)
fig_starlink.update_yaxes(title_text='Trust Weight', row=2, col=1)
fig_starlink.update_xaxes(title_text='Time [s]', row=2, col=1)
fig_starlink.update_layout(template=PLOT_TEMPLATE, height=690, title='Post-Blackout Reacquisition Dynamics')
fig_starlink.show()


## Monte-Carlo Dispersion Analysis (360 Runs)

Dispersions applied (representative):

- Entry flight-path angle uncertainty: `±0.5°` (sampled and clipped)
- Initial IMU bias dispersion
- IMU noise-level dispersion

We plot mean ± `3σ` envelopes for position and attitude error and report 95th-percentile metrics.


In [ ]:
mc = run_monte_carlo_dispersion(
    df,
    blackout_start_s=blackout_start_s,
    blackout_end_s=blackout_end_s,
    n_runs=MC_RUNS,
    seed=2026,
)

fig_mc = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=(
        'Position Error Mean ± 3σ',
        'Attitude Error Mean ± 3σ',
    ),
)

add_sigma_band(
    fig_mc, mc['time_s'], mc['stats']['inertial']['pos_mean'], mc['stats']['inertial']['pos_sigma'],
    'Inertial', COLORS['inertial'], row=1, col=1, showlegend=True
)
add_sigma_band(
    fig_mc, mc['time_s'], mc['stats']['ekf']['pos_mean'], mc['stats']['ekf']['pos_sigma'],
    'EKF', COLORS['ekf'], row=1, col=1, showlegend=True
)
add_sigma_band(
    fig_mc, mc['time_s'], mc['stats']['dsfb']['pos_mean'], mc['stats']['dsfb']['pos_sigma'],
    'DSFB', COLORS['dsfb'], row=1, col=1, showlegend=True
)

add_sigma_band(
    fig_mc, mc['time_s'], mc['stats']['inertial']['att_mean'], mc['stats']['inertial']['att_sigma'],
    'Inertial (att)', COLORS['inertial'], row=1, col=2, showlegend=False
)
add_sigma_band(
    fig_mc, mc['time_s'], mc['stats']['ekf']['att_mean'], mc['stats']['ekf']['att_sigma'],
    'EKF (att)', COLORS['ekf'], row=1, col=2, showlegend=False
)
add_sigma_band(
    fig_mc, mc['time_s'], mc['stats']['dsfb']['att_mean'], mc['stats']['dsfb']['att_sigma'],
    'DSFB (att)', COLORS['dsfb'], row=1, col=2, showlegend=False
)

for c in (1, 2):
    fig_mc.add_vrect(x0=blackout_start_s, x1=blackout_end_s, fillcolor=BLACKOUT_SHADE, line_width=0, row=1, col=c)

fig_mc.update_xaxes(title_text='Time [s]', row=1, col=1)
fig_mc.update_xaxes(title_text='Time [s]', row=1, col=2)
fig_mc.update_yaxes(title_text='Position Error [m]', type='log', row=1, col=1)
fig_mc.update_yaxes(title_text='Attitude Error [deg]', row=1, col=2)
fig_mc.update_layout(template=PLOT_TEMPLATE, height=500, title=f'Monte-Carlo Dispersion Envelopes ({mc["n_runs"]} runs)')
fig_mc.show()

mc_table = build_mc_95_table(mc)
display(mc_table.round(3))
display(Markdown(
    '*Table 2: Monte-Carlo 95th-percentile dispersion summary (360 runs with dispersed entry flight-path angle, IMU bias, and sensor noise).*'
))


## Interactive Sliders for Slew Threshold and ρ

Use sliders to rerun the DSFB configuration and inspect summary metrics quickly.


In [ ]:
def quick_sweep(rho=0.97, slew_threshold=32.0):
    s, d = run_demo(rho=rho, slew_threshold=slew_threshold, output_dir='output-dsfb-starship')
    b0, b1 = get_blackout_bounds(d)
    blackout_duration = 0.0 if b0 is None else (b1 - b0)

    table = pd.DataFrame({
        'Estimator': ['Pure Inertial', 'Simple EKF', 'DSFB'],
        'RMSE Position [m]': [s['inertial']['rmse_position_m'], s['ekf']['rmse_position_m'], s['dsfb']['rmse_position_m']],
        'RMSE Attitude [deg]': [s['inertial']['rmse_attitude_deg'], s['ekf']['rmse_attitude_deg'], s['dsfb']['rmse_attitude_deg']],
        'Final Position Error [m]': [s['inertial']['final_position_error_m'], s['ekf']['final_position_error_m'], s['dsfb']['final_position_error_m']],
    })

    print(
        f"Blackout duration: {blackout_duration:.1f} s | "
        f"rho={rho:.3f} | slew_threshold={slew_threshold:.1f}"
    )
    display(table.round(3))

widgets.interact(
    quick_sweep,
    rho=widgets.FloatSlider(value=0.97, min=0.90, max=0.995, step=0.005),
    slew_threshold=widgets.FloatSlider(value=32.0, min=15.0, max=60.0, step=1.0),
)


## Conclusions

<div style="border-left: 6px solid #FFD166; background: #2B2B2B; padding: 10px 14px; margin-bottom: 10px;">
<b>Illustrative Result:</b> DSFB trust adaptation attenuates faulty IMU channels during transients, and post-blackout Starlink-like reacquisition rapidly pulls the navigation estimate back toward truth.
</div>

- The trajectory and blackout timing are tuned to be representative of publicly released Starship IFT webcast behavior (2025) while remaining an open, reproducible simulation.
- Representative aerodynamic coefficients and heating relations from public NASA/NACA/AIAA references provide physically plausible 6-DoF re-entry dynamics.
- Monte-Carlo dispersion (360 runs) with flight-path-angle, IMU bias, and noise uncertainty provides robustness envelopes (`mean ± 3σ`) and paper-ready 95th-percentile tables.
- The Starlink reacquisition channel explicitly demonstrates DSFB trust re-weighting at blackout exit and rapid state correction dynamics.
- DSFB demonstrates bounded, trust-adaptive behavior that could complement existing navigation architectures during the critical plasma blackout phase.

> **Disclaimer (Important):** This is an illustrative simulation using representative physics and parameters. It is not calibrated to proprietary SpaceX models or actual flight data. Absolute performance numbers are not predictive of any specific vehicle.
